[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gcmatos/python-para-geociencias/blob/master/notebooks/3.1%20Limpeza%20e%20transformação%20de%20dados.ipynb)

Ctrl/Cmd + click para abrir em uma nova aba do navegador web e utilizar o Google Colab para rodar o tutorial.

# Limpeza e transformação de dados

__O que iremos aprender__
- Ferramentas para tratar valores nulos
- Manipulação de dados
- Operações de agrupamento
- Gerar sumários estatísticos por grupos de dados

## Configuração de ambiente

In [0]:
import numpy as np
import pandas as pd

In [0]:
# Números aleatórios
np.random.seed(0)

## Limpeza de dados
As tarefas de detecção e correção de registros inválidos em conjuntos de dados são conhecidas genericamente como tarefas de limpeza, que podem ser feitas de forma interativa (*e.g. *Excel) ou programática (*e.g. * R, Python). Estas falhas no registro podem ser causadas no momento da aquisição por defeitos em equipamentos ou por falhas humanas como erros de digitação.

### Valores nulos

In [0]:
s = pd.Series([1, np.nan, 3.5, np.nan, 7])
s

In [0]:
# Total de valores nulos
s.isnull().sum()

In [0]:
# Remover valores nulos
s.dropna()
# s[s.notnull()]

In [0]:
df = pd.DataFrame([[1., 6.5, 3.], 
                   [2., np.nan, np.nan],
                   [np.nan, np.nan, np.nan], 
                   [np.nan, 7.5, 4]])
df

In [0]:
# Total de valores nulos por linhas ou por colunas
df.isnull().sum()
# df.isnull().sum(axis=1)

In [0]:
# Remoer valores nulos
df.dropna()
# df.dropna(axis=1)

In [0]:
# Substituir valores nulos por constantes
# df.fillna(-9999)
df.fillna(df.mean())

### Substituir valores

In [0]:
df = pd.Series([1., -999., 2., -999., -1000., 3.])
df

In [0]:
df.replace(-999, np.nan)
# df.replace([-999, -1000], np.nan)
# df.replace([-999, -1000], [np.nan, 0])
# df.replace({-999: np.nan, -1000: 0})

### *Outliers*
Registros de valores fora de limites pré-estabelecidos como aceitáveis são denomidados *outliers* e podem causar problemas em sumários estatísticos e modelos preditivos.

In [0]:
df = pd.DataFrame(np.random.randn(1000, 4))
df

In [0]:
df.describe()

In [0]:
# Selecionar valores fora do intervalo [-3, 3] em uma coluna
col = df[2]
col[np.abs(col) > 3]

In [0]:
# Selecionar linhas com valores fora do intervato [-3, 3]
df[(np.abs(df) > 3).any(1)]

In [0]:
# Substituir valores fora do intervalo por valor constante com o mesmo sinal que o valor original
df[np.abs(df) > 3] = np.sign(df) * 3
# df[(np.abs(df) == 3).any(1)]

### Remover duplicatas

In [0]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [0]:
data.duplicated()
# data.drop_duplicates()
# data.drop_duplicates(['k1'])
# data.drop_duplicates(['k1', 'k2'], keep='first')

## Transformação com funções de mapeamento


__Exemplo:__

Para exemplificar a utilização da função `map()`, criamos um dataset que contém uma coluna com medidas de porosidade efetiva e outra com a litologia. Vamos criar uma nova coluna com a categoria 'rocktype', utilizando a função de mapeamento, que irá informar quais amostras são de rocha reservatório ou não reservatório.

In [0]:
data = pd.DataFrame({'rock': ['Sandstone', 'Sandstone', 
                              'Sandstone', 'Shale', 'Wackstone', 
                              'Siltstone', 'Sandstone', 
                              'Conglomerate', 'Limestone'],
                     'porosity': [0.2, 0.25, 0.1, 0.03, 0.07, 0.08, 0.3, 0.15, 0.36]})
data.head()

In [0]:
# Definir dicionário mapa de tipos de reservatório
rock_type = {
  'sandstone': 'reservoir',
  'conglomerate': 'reservoir',
  'siltstone': 'non-reservoir',
  'wackstone': 'non-reservoir',
  'limestone': 'reservoir',
  'shale': 'non-reservoir'
}

In [0]:
# Aplicar função diretamente nos elementos da coluna 'rock'
data['rock'].str.lower().map(rock_type)
# data['rock_type'] = data['rock'].map(rock_type)
# data['rock_type'] = data['rock'].str.lower().map(rock_type)
# data.head()

In [0]:
# Outra forma de resolver o mapeamento de valores
lowercased = data['rock'].str.lower()
# lowercased

# Criar nova coluna 'rock_type' com função map()
data['rock_type'] = lowercased.map(rock_type)
data.head()

## Discretização
Objetos pandas podem ser discretizados em intervalos de valores com a função `pandas.cut`. Esta função cria uma variável do tipo `CategoricalDtype`

__Exemplo__

Para exemplificar a utilização da função `pd.cut()`, vamos criar um dataset que contém uma lista com 100 mediadas de Gamma Ray (API). Em seguida iremos criar 4 categorias de representam intervalos de valores.

In [0]:
gamma = pd.DataFrame(abs(np.random.randn(100)) * 60, columns=['Gamma Ray'])
# gamma.sample(5)
gamma.plot.hist();

In [0]:
# Separar em grupos de intervalos regularmente espaçados
gamma_cut = pd.cut(gamma['Gamma Ray'], 4)
# gamma_cut = pd.cut(gamma['Gamma Ray'], 4, labels=['low', 'medium', 'high', 'extreme'])

gamma_cut.head()
# gamma_cut.dtype
# pd.value_counts(gamma_cut)
# gamma_cut.value_counts()
# gamma_cut.values

In [0]:
# Definir os intervalos de cutoff
cutoff = [0, 50, 100, 150, 200]

gamma_cut = pd.cut(gamma['Gamma Ray'], cutoff)

# gamma_cut.head()
# gamma_cut.dtype
# gamma_cut.value_counts().plot(kind='barh')
# gamma_cut.values

In [0]:
# Particionar em quartil
gamma_cut_q = pd.qcut(gamma['Gamma Ray'], 4)
# gamma_cut_q = pd.qcut(gamma['Gamma Ray'], [0, 0.1, 0.5, 0.9, 1.])

# gamma_cut_q.head()
gamma_cut_q.value_counts()
# gamma_cut_q.values

## Manipulação

### Hierarquia de índices
Objetos pandas podem conter uma ou mais colunas com índices em diferentes níveis hierárquicos. Esse aspectos é extremamente útil para geociências, pois isso torna possível que sejam utilizadas colunas com coordenadas (x, y) e identificação de pontos como índices em planilhas de aquisição de dados.

In [0]:
s = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
s

In [0]:
s.index

In [0]:
s['b']
# data.loc[['b', 'd']]

In [0]:
# Desmontar índice hierárquico
s.unstack()

### Converter colunas em índices hierárquicos

In [0]:
df = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
df

In [0]:
df2 = df.set_index(['c', 'd'])
# df2 = df.set_index(['c', 'd'], drop=False)
df2

In [0]:
df2.reset_index()

### Estatística por índices hiearárquicos


__Exemplo:__


In [0]:
# DataFrame de dados de fácies e rochas
df = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['facies1', 'facies1', 'facies2'],
                              ['Sand', 'Shale', 'Limestone']])
df

In [0]:
# Nomear índices
df.index.names = ['point', 'sample']
df.columns.names = ['Facies', 'Rock']
df

In [0]:
df['facies2']

In [0]:
df.sum(level= 'key1')
# df.sum(level='Facies', axis=1)

## Combinar data sets
Os dois métodos pandas para combinar objetos que serão tratados neste curso são:

- **`pandas.concat`** empilha objetos pandas concatenando os mesmo ao longo de um eixo.

- **`pandas.merge`** que conecta linhas em DataFrames baseados em uma ou mais chaves como em uma base realcional (SQL). 


### Tabela de opções de combinação relacional


Método pd. merge| SQL | Descrição
--- | ---
how=`'inner'` | `INNER JOIN` | Combinar usando apenas chaves contidas (interseções) entre tabelas
how=`'left'` | `LEFT OUTER JOIN` | Combinar usando apenas chaves contidas na tabela da esquerda
how=`'right'` | `RIGHT OUTER JOIN` | Combinar usando apenas chaves contidas na tabela da direita
how=`'outer'` | `FULL OUTER JOIN` | Combinar usando todas as chaves contidas nas tabelas

### `merge`

In [0]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})
left

In [0]:
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
right

In [0]:
pd.merge(left, right, on=['key1', 'key2'])
# pd.merge(left, right, how='inner', on=['key1', 'key2'])

### `concatenate`

In [0]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [0]:
# COncatenar por eixo
pd.concat([s1, s2, s3], axis=1)

In [0]:
s4 = pd.concat([s1, s3])
s4

In [0]:
pd.concat([s1, s4], axis=1, join='inner')

In [0]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

In [0]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1

In [0]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

In [0]:
# COncatenar ignorando índices
pd.concat([df1, df2])
# pd.concat([df1, df2], ignore_index=True)

## Rearranjos e modificações

Métodos:
#### **`stack`** 
> Rotaciona (pivot) de colunas para linhas
#### **`unstack`**
> Rotaciona  (pivot) de linhas para colunas

In [0]:
df = pd.DataFrame(np.arange(6).reshape((2, 3)),
                  index=pd.Index(['A', 'B'], name='index'),
                  columns=pd.Index(['one', 'two', 'three'],
                  name='columns'))
df

In [0]:
df.stack()
# df.stack().unstack()

In [0]:
# Unstack com especificação de índices
df_stacked = df.stack()

df_stacked.unstack('index')

# Agrupamento

As principais operações de agrupamento são:
- Fragmentar (split)
- Estatísticas por subgrupo
- Aplicação de métodos em grupos espeíficos dentro de uma tabela
- Computar tabelas pivot

![](https://i.stack.imgur.com/sgCn1.jpg)

Exemplo de agrupamento com a metodologia *split-apply-combine* utilizando o método `groupby`.

## GroupBy

### *split-apply-combine*

In [0]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

In [0]:
# split
grouped = df['data1'].groupby(df['key1'])
# grouped

# apply-combine
grouped.mean()

In [0]:
# split-apply-combine com múltiplas chaves
means = df.groupby([df['key1'], df['key2']]).mean()
# means = df['data1'].groupby([df['key1'], df['key2']]).mean()

means

In [0]:
means.unstack()

In [0]:
# Gerar DataFrame com GroupBy
df_grouped = df.groupby(['key1', 'key2'])[['data2']].mean()
type(df_grouped)

In [0]:
# Gerar Series com GroupBy
s_grouped = df.groupby(['key1', 'key2'])['data2'].mean()
type(s_grouped)

### Iterações em grupos

In [0]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

In [0]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

In [0]:
df.dtypes

In [0]:
grouped = df.groupby(df.dtypes, axis=1)

for dtype, group in grouped:
    print(dtype)
    print(group)

### Agrupamento com dicionários de valores

In [0]:
df = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['line 1', 'line 2', 'line 3', 'line 4', 'line 5'])
df

In [0]:
facies = {'a': 'type 1', 'b': 'type 1', 'c': 'type 1',
          'd': 'type 2', 'e': 'type 2'}

In [0]:
# Agrupar por colunas
df.groupby(facies, axis=1).mean()

## Agregando valores
Agregar valores significa gerar valores escalares a partir da redução de arranjos de números. Algumas das funções mais utilizadas para agregar valores são:
- `sum` 
- `mean`
- `median`
- `min`, `max`

In [0]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

In [0]:
# Agrupar por quantil
df.groupby('key1')['data1'].quantile(0.9)

In [0]:
# Agrupar usando funções

def interval(arr):
    return arr.max() - arr.min()


df.groupby('key1').agg(interval)

In [0]:
df.groupby('key1').describe()

In [0]:
grouped = df.groupby('key1')
# grouped = df.groupby('key1', as_index=False)
# grouped = df.groupby('key1', group_keys=False)

grouped.agg('mean')

In [0]:
grouped.agg(['mean', 'median', interval])

## Tabelas Pivot e *cross-tabulations*
Tabelas pivot geram sumários estatísticos e *cross-tabulations *geram sumários com frequências de valores.

In [0]:
df

In [0]:
# Utilizando tablea pivot
df.pivot_table(index='key1')
# df.pivot_table(index='key1', columns='key2')

In [0]:
pd.crosstab(df.key1, df.key2, margins=True)

# Referências

- [Limpeza de dados](https://en.wikipedia.org/wiki/Data_cleansing)